<a href="https://colab.research.google.com/github/Martoto/BigData_ForecastHackathon/blob/main/silicionauta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%cd Users/daniel.sant.salles/

/mnt/batch/tasks/shared/LS_root/mounts/clusters/memoryman/code/Users/daniel.sant.salles


In [4]:
%pip install pandas pyarrow seaborn plotly

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.8 MB 4.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.8 MB 2.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.8 MB 2.4 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.8 MB 2.6 MB/s eta 0:00:03
   ---------- ----------------------------- 2.6/9.8 MB 2.6 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/9.8 MB 2.7 MB/s eta 0:00:03
   ---------------- ----------------------- 3.9/9.8 MB 2.7 MB/s eta 0:00:03
   ------------------- -------------------- 4.7/9.8 MB 2.9 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.8 MB 2.8 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/9.8 MB 2.9 MB/s eta 0:00:02
   -------------------------- ------------- 6.6/9.8 MB 2.9 MB/s eta 0:00:02
   ----------------------------- ---------- 7.3/9.8 MB 2.9 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import requests

file_path = '/content/hackathon_2025_templates.zip'
file_id = '1Ed-nqUyCT0z-T0AvHwLMsGqHYqjw2248' # This is the file ID

if not os.path.exists(file_path):
    print(f'{file_path} not found. Downloading...')

    # Construct the direct download URL
    download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

    try:
        response = requests.get(download_url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f'{file_path} downloaded successfully.')

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
        print("Please ensure the file is shared publicly or with 'Anyone with the link'.")

else:
    print(f'{file_path} found.')

/content/hackathon_2025_templates.zip found.


In [8]:
import zipfile
import os

zip_path = 'hackathon_2025_templates.zip'
extract_dir = './data/'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f'Extracted {zip_path} to {extract_dir}')
else:
    print(f'{zip_path} not found in the current directory.')

Extracted hackathon_2025_templates.zip to ./data/


In [9]:
import pandas as pd
file_path = './data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
df_pdv = pd.read_parquet(file_path)
display(df_pdv.head())

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [1]:
import pandas as pd
file_path = './data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
df_transactions = pd.read_parquet(file_path)
display(df_transactions.head())

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


In [2]:
import pandas as pd
import pyarrow.parquet as pq

# Get unique product IDs from transactions for filtering
valid_product_ids = set(df_transactions['internal_product_id'].unique())
print(f"Found {len(valid_product_ids):,} unique product IDs in transactions")

file_path = './data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet'
table = pq.ParquetFile(file_path)

# Collect filtered chunks
filtered_chunks = []
total_rows_processed = 0
total_rows_kept = 0

for batch in table.iter_batches(batch_size=5000):
    df_chunk = batch.to_pandas()
    total_rows_processed += len(df_chunk)
    
    # Filter to keep only products that exist in transactions
    filtered_chunk = df_chunk[df_chunk['produto'].isin(valid_product_ids)]
    total_rows_kept += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        filtered_chunks.append(filtered_chunk)

# Combine all filtered chunks
df_products = pd.concat(filtered_chunks, ignore_index=True) if filtered_chunks else pd.DataFrame()

print(f"Total rows processed: {total_rows_processed:,}")
print(f"Total rows kept: {total_rows_kept:,}")
print(f"Filtering efficiency: {(total_rows_kept/total_rows_processed)*100:.2f}%")
print(f"Final products dataframe shape: {df_products.shape}")
print("\nFirst few rows of filtered products:")
display(df_products.head())

Found 7,092 unique product IDs in transactions
Total rows processed: 192,356,316
Total rows kept: 192,356,316
Filtering efficiency: 100.00%
Final products dataframe shape: (192356316, 8)

First few rows of filtered products:
Total rows processed: 192,356,316
Total rows kept: 192,356,316
Filtering efficiency: 100.00%
Final products dataframe shape: (192356316, 8)

First few rows of filtered products:


,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery


In [ ]:
# Verification of the filtering results
print("🔍 FILTERING VERIFICATION")
print("="*50)

# Check unique products in filtered dataset
unique_products_filtered = df_products['produto'].nunique()
print(f"Unique products in filtered dataset: {unique_products_filtered:,}")

# Check if all transaction products are covered
transaction_products = set(df_transactions['internal_product_id'].unique())
filtered_products = set(df_products['produto'].unique())

print(f"Unique products in transactions: {len(transaction_products):,}")
print(f"Unique products in filtered products: {len(filtered_products):,}")

# Check intersection
intersection = transaction_products.intersection(filtered_products)
print(f"Products in both datasets: {len(intersection):,}")

# Check if there are products in transactions not in products
missing_in_products = transaction_products - filtered_products
if missing_in_products:
    print(f"⚠️  Products in transactions but missing in products: {len(missing_in_products)}")
    print(f"Examples: {list(missing_in_products)[:10]}")
else:
    print("✅ All transaction products are covered in the products dataset")

# Check if there are products in products not in transactions
missing_in_transactions = filtered_products - transaction_products
if missing_in_transactions:
    print(f"⚠️  Products in products but missing in transactions: {len(missing_in_transactions)}")
    print(f"Examples: {list(missing_in_transactions)[:10]}")
else:
    print("✅ All products have corresponding transactions")

print(f"\n📊 Data consistency: {'Perfect match!' if len(missing_in_products) == 0 and len(missing_in_transactions) == 0 else 'Some mismatches found'}")

1- pdv	premise	categoria_pdv	zipcode 
2- internal_store_id	internal_product_id	distributor_id	transaction_date	reference_date	quantity	gross_value	net_value	gross_profit	discount	taxes
3- produto categoria descricao tipos label subcategoria marca fabricante               


## Visão Geral da Estrutura dos Dados

### 📊 Dataset 1: Ponto de Venda (PDV)
| Coluna | Descrição |
|--------|-----------|
| `pdv` | Identificador do Ponto de Venda |
| `premise` | Informações da loja/estabelecimento |
| `categoria_pdv` | Classificação da categoria do PDV |
| `zipcode` | Código postal/CEP |

### 💳 Dataset 2: Transações
| Coluna | Descrição |
|--------|-----------|
| `internal_store_id` | Identificador interno da loja |
| `internal_product_id` | Identificador interno do produto |
| `distributor_id` | Identificador do distribuidor |
| `transaction_date` | Data da transação |
| `reference_date` | Data de referência |
| `quantity` | Quantidade da transação |
| `gross_value` | Valor bruto da transação |
| `net_value` | Valor líquido da transação |
| `gross_profit` | Lucro bruto |
| `discount` | Desconto aplicado |
| `taxes` | Valor dos impostos |

### 🛍️ Dataset 3: Produtos
| Coluna | Descrição |
|--------|-----------|
| `produto` | Identificador do produto |
| `categoria` | Categoria do produto |
| `descricao` | Descrição do produto |
| `tipos` | Tipos do produto |
| `label` | Rótulo do produto |
| `subcategoria` | Subcategoria do produto |
| `marca` | Marca |
| `fabricante` | Fabricante |


# 🚀 EXPLORATORY DATA ANALYSIS - HACKATHON 2025
## Going HAM on the Data! 💪

Let's dive deep into this retail dataset and uncover insights for forecasting!

In [5]:
# Import comprehensive analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("🚀 Libraries loaded successfully! Let's go HAM on this data!")

🚀 Libraries loaded successfully! Let's go HAM on this data!


In [ ]:
# 📊 LOAD ALL DATASETS
print("🔄 Loading all three datasets...")

# Dataset 1: PDV (Points of Sale)
pdv_file = './data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
df_pdv = pd.read_parquet(pdv_file)
print(f"✅ PDV Dataset loaded: {df_pdv.shape}")

# Dataset 2: Transactions
transactions_file = './data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
df_transactions = pd.read_parquet(transactions_file)
print(f"✅ Transactions Dataset loaded: {df_transactions.shape}")

# Dataset 3: Products (using chunked reading for large file)
products_file = './data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet'
df_products = pd.read_parquet(products_file)
print(f"✅ Products Dataset loaded: {df_products.shape}")

print(f"\n🎯 Total datasets loaded: 3")
print(f"📈 Combined data points: {df_pdv.shape[0] + df_transactions.shape[0] + df_products.shape[0]:,}")

🔄 Loading all three datasets...
✅ PDV Dataset loaded: (14419, 4)
✅ Transactions Dataset loaded: (6560698, 11)


In [2]:
# 🔍 DATASET STRUCTURE ANALYSIS - Going Deep!
print("="*80)
print("🔍 DEEP DIVE INTO DATASET STRUCTURES")
print("="*80)

datasets = {
    'PDV': df_pdv,
    'Transactions': df_transactions,
    'Products': df_products
}

for name, df in datasets.items():
    print(f"\n📊 {name.upper()} DATASET ANALYSIS:")
    print(f"   📏 Shape: {df.shape}")
    print(f"   📋 Columns: {list(df.columns)}")
    print(f"   🏷️  Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"      {col}: {dtype}")
    print(f"   💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   🕳️  Missing Values: {df.isnull().sum().sum()}")
    print(f"   🔢 Unique Values per Column:")
    for col in df.columns:
        unique_count = df[col].nunique()
        print(f"      {col}: {unique_count:,}")
    print("-" * 60)

🔍 DEEP DIVE INTO DATASET STRUCTURES


NameError: name 'df_pdv' is not defined

In [ ]:
# 🧹 DATA QUALITY ASSESSMENT - The Deep Clean!
print("="*80)
print("🧹 DATA QUALITY ASSESSMENT")
print("="*80)

def analyze_data_quality(df, name):
    print(f"\n🔍 {name} QUALITY REPORT:")
    
    # Missing values analysis
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    
    print(f"📊 Missing Values Analysis:")
    for col in df.columns:
        if missing_values[col] > 0:
            print(f"   {col}: {missing_values[col]:,} ({missing_percent[col]:.2f}%)")
    
    # Duplicates analysis
    duplicates = df.duplicated().sum()
    print(f"🔄 Duplicate Rows: {duplicates:,} ({(duplicates/len(df)*100):.2f}%)")
    
    # Numerical columns analysis
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 0:
        print(f"📈 Numerical Columns Analysis:")
        for col in numerical_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = ((df[col] < (q1 - 1.5 * iqr)) | (df[col] > (q3 + 1.5 * iqr))).sum()
            print(f"   {col}: Min={df[col].min():.2f}, Max={df[col].max():.2f}, Outliers={outliers:,}")
    
    print("-" * 60)

# Analyze each dataset
for name, df in datasets.items():
    analyze_data_quality(df, name)

In [ ]:
# 💰 SALES TRANSACTION ANALYSIS - The Money Maker!
print("="*80)
print("💰 SALES TRANSACTION ANALYSIS")
print("="*80)

# Convert date columns to datetime
if 'transaction_date' in df_transactions.columns:
    df_transactions['transaction_date'] = pd.to_datetime(df_transactions['transaction_date'])
if 'reference_date' in df_transactions.columns:
    df_transactions['reference_date'] = pd.to_datetime(df_transactions['reference_date'])

# Time period analysis
print(f"📅 Transaction Date Range:")
if 'transaction_date' in df_transactions.columns:
    print(f"   From: {df_transactions['transaction_date'].min()}")
    print(f"   To: {df_transactions['transaction_date'].max()}")
    print(f"   Duration: {(df_transactions['transaction_date'].max() - df_transactions['transaction_date'].min()).days} days")

# Financial metrics overview
numerical_cols = ['quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
available_cols = [col for col in numerical_cols if col in df_transactions.columns]

print(f"\n💵 Financial Metrics Summary:")
if available_cols:
    for col in available_cols:
        print(f"   {col}:")
        print(f"      Total: ${df_transactions[col].sum():,.2f}")
        print(f"      Average: ${df_transactions[col].mean():.2f}")
        print(f"      Median: ${df_transactions[col].median():.2f}")
        print(f"      Std Dev: ${df_transactions[col].std():.2f}")

# Top performing stores and products
print(f"\n🏪 Top 10 Stores by Total Revenue:")
if 'internal_store_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    top_stores = df_transactions.groupby('internal_store_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for store_id, revenue in top_stores.items():
        print(f"   Store {store_id}: ${revenue:,.2f}")

print(f"\n🛍️ Top 10 Products by Total Sales:")
if 'internal_product_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    top_products = df_transactions.groupby('internal_product_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for product_id, revenue in top_products.items():
        print(f"   Product {product_id}: ${revenue:,.2f}")

In [ ]:
# 📈 TIME SERIES ANALYSIS - The Time Machine!
print("="*80)
print("📈 TIME SERIES ANALYSIS")
print("="*80)

if 'transaction_date' in df_transactions.columns:
    # Extract time components
    df_transactions['year'] = df_transactions['transaction_date'].dt.year
    df_transactions['month'] = df_transactions['transaction_date'].dt.month
    df_transactions['day'] = df_transactions['transaction_date'].dt.day
    df_transactions['weekday'] = df_transactions['transaction_date'].dt.day_name()
    df_transactions['week'] = df_transactions['transaction_date'].dt.isocalendar().week
    
    # Daily sales analysis
    daily_sales = df_transactions.groupby('transaction_date').agg({
        'gross_value': ['sum', 'mean', 'count'],
        'quantity': 'sum'
    }).round(2)
    
    print(f"📊 Daily Sales Statistics:")
    print(f"   Total Days with Transactions: {len(daily_sales)}")
    print(f"   Average Daily Revenue: ${daily_sales[('gross_value', 'sum')].mean():,.2f}")
    print(f"   Best Day Revenue: ${daily_sales[('gross_value', 'sum')].max():,.2f}")
    print(f"   Worst Day Revenue: ${daily_sales[('gross_value', 'sum')].min():,.2f}")
    
    # Weekly patterns
    weekly_patterns = df_transactions.groupby('weekday')['gross_value'].sum().sort_values(ascending=False)
    print(f"\n📅 Weekly Sales Patterns:")
    for day, revenue in weekly_patterns.items():
        print(f"   {day}: ${revenue:,.2f}")
    
    # Monthly trends
    if 'gross_value' in df_transactions.columns:
        monthly_trends = df_transactions.groupby(['year', 'month'])['gross_value'].sum()
        print(f"\n🗓️ Monthly Revenue Trends:")
        for (year, month), revenue in monthly_trends.items():
            print(f"   {year}-{month:02d}: ${revenue:,.2f}")

# Seasonality detection
print(f"\n🌟 Seasonality Insights:")
if 'month' in df_transactions.columns:
    seasonal_pattern = df_transactions.groupby('month')['gross_value'].mean().round(2)
    peak_month = seasonal_pattern.idxmax()
    low_month = seasonal_pattern.idxmin()
    print(f"   Peak Sales Month: {peak_month} (${seasonal_pattern[peak_month]:,.2f} avg)")
    print(f"   Lowest Sales Month: {low_month} (${seasonal_pattern[low_month]:,.2f} avg)")

In [ ]:
# 🛍️ PRODUCT PORTFOLIO ANALYSIS - The Product Universe!
print("="*80)
print("🛍️ PRODUCT PORTFOLIO ANALYSIS")
print("="*80)

# Product categories analysis
print("📦 Product Categories Overview:")
if 'categoria' in df_products.columns:
    category_counts = df_products['categoria'].value_counts()
    print(f"   Total Categories: {len(category_counts)}")
    print("   Top 10 Categories:")
    for cat, count in category_counts.head(10).items():
        print(f"      {cat}: {count:,} products")

# Brand analysis
print(f"\n🏷️ Brand Analysis:")
if 'marca' in df_products.columns:
    brand_counts = df_products['marca'].value_counts()
    print(f"   Total Brands: {len(brand_counts)}")
    print("   Top 10 Brands:")
    for brand, count in brand_counts.head(10).items():
        print(f"      {brand}: {count:,} products")

# Manufacturer analysis
print(f"\n🏭 Manufacturer Analysis:")
if 'fabricante' in df_products.columns:
    mfg_counts = df_products['fabricante'].value_counts()
    print(f"   Total Manufacturers: {len(mfg_counts)}")
    print("   Top 10 Manufacturers:")
    for mfg, count in mfg_counts.head(10).items():
        print(f"      {mfg}: {count:,} products")

# Product types analysis
print(f"\n🎯 Product Types Distribution:")
if 'tipos' in df_products.columns:
    type_counts = df_products['tipos'].value_counts()
    print(f"   Total Types: {len(type_counts)}")
    for ptype, count in type_counts.head(15).items():
        print(f"      {ptype}: {count:,} products")

In [ ]:
# 🗺️ GEOGRAPHICAL ANALYSIS - The Map Masters!
print("="*80)
print("🗺️ GEOGRAPHICAL ANALYSIS")
print("="*80)

# PDV location analysis
if 'zipcode' in df_pdv.columns:
    zipcode_analysis = df_pdv['zipcode'].value_counts()
    print(f"📍 Zipcode Distribution:")
    print(f"   Total Unique Zipcodes: {len(zipcode_analysis)}")
    print("   Top 15 Zipcodes by Store Count:")
    for zip_code, count in zipcode_analysis.head(15).items():
        print(f"      {zip_code}: {count:,} stores")

# PDV category analysis
if 'categoria_pdv' in df_pdv.columns:
    pdv_categories = df_pdv['categoria_pdv'].value_counts()
    print(f"\n🏪 PDV Categories:")
    print(f"   Total Categories: {len(pdv_categories)}")
    for category, count in pdv_categories.items():
        print(f"      {category}: {count:,} stores")

# Store premise analysis
if 'premise' in df_pdv.columns:
    premise_analysis = df_pdv['premise'].value_counts()
    print(f"\n🏢 Store Premise Types:")
    print(f"   Total Premise Types: {len(premise_analysis)}")
    for premise, count in premise_analysis.head(10).items():
        print(f"      {premise}: {count:,} stores")

print("-" * 60)

In [ ]:
# 📈 ADVANCED VISUALIZATIONS - The Eye Candy!
print("="*80)
print("📈 CREATING KILLER VISUALIZATIONS")
print("="*80)

# Create a subplot figure for comprehensive analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Daily Sales Trend', 'Weekly Pattern', 'Top Categories', 'Revenue Distribution'),
    specs=[[{"secondary_y": True}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "histogram"}]]
)

# 1. Daily sales trend
if 'transaction_date' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    daily_sales = df_transactions.groupby('transaction_date')['gross_value'].sum().reset_index()
    fig.add_trace(
        go.Scatter(x=daily_sales['transaction_date'], y=daily_sales['gross_value'],
                  mode='lines+markers', name='Daily Sales'),
        row=1, col=1
    )

# 2. Weekly patterns
if 'weekday' in df_transactions.columns:
    weekly_data = df_transactions.groupby('weekday')['gross_value'].sum().reset_index()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_data['weekday'] = pd.Categorical(weekly_data['weekday'], categories=weekday_order, ordered=True)
    weekly_data = weekly_data.sort_values('weekday')
    
    fig.add_trace(
        go.Bar(x=weekly_data['weekday'], y=weekly_data['gross_value'],
               name='Weekly Sales', marker_color='lightblue'),
        row=1, col=2
    )

# 3. Top product categories (if we can link products to transactions)
if 'categoria' in df_products.columns:
    top_categories = df_products['categoria'].value_counts().head(8)
    fig.add_trace(
        go.Pie(labels=top_categories.index, values=top_categories.values,
               name="Categories"),
        row=2, col=1
    )

# 4. Revenue distribution
if 'gross_value' in df_transactions.columns:
    fig.add_trace(
        go.Histogram(x=df_transactions['gross_value'], name='Revenue Distribution',
                    nbinsx=50, marker_color='orange'),
        row=2, col=2
    )

# Update layout
fig.update_layout(height=800, showlegend=True, 
                  title_text="🚀 Comprehensive Retail Analytics Dashboard")
fig.show()

print("✅ Interactive dashboard created!")

In [ ]:
# 🔗 CORRELATION ANALYSIS - Finding Hidden Relationships!
print("="*80)
print("🔗 CORRELATION ANALYSIS")
print("="*80)

# Financial metrics correlation
financial_cols = ['quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
available_financial = [col for col in financial_cols if col in df_transactions.columns]

if len(available_financial) > 1:
    correlation_matrix = df_transactions[available_financial].corr()
    
    # Create correlation heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title('💰 Financial Metrics Correlation Matrix', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("🔍 Key Correlations:")
    # Find strongest correlations
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:  # Strong correlation
                col1, col2 = correlation_matrix.columns[i], correlation_matrix.columns[j]
                print(f"   {col1} ↔ {col2}: {corr_val:.3f}")

# Distributor performance analysis
if 'distributor_id' in df_transactions.columns and 'gross_value' in df_transactions.columns:
    distributor_performance = df_transactions.groupby('distributor_id').agg({
        'gross_value': ['sum', 'mean', 'count'],
        'quantity': 'sum',
        'gross_profit': 'sum' if 'gross_profit' in df_transactions.columns else 'mean'
    }).round(2)
    
    print(f"\n🚚 Top 10 Distributors by Revenue:")
    top_distributors = df_transactions.groupby('distributor_id')['gross_value'].sum().sort_values(ascending=False).head(10)
    for dist_id, revenue in top_distributors.items():
        transactions_count = df_transactions[df_transactions['distributor_id'] == dist_id].shape[0]
        print(f"   Distributor {dist_id}: ${revenue:,.2f} ({transactions_count:,} transactions)")

print("-" * 60)